In [ ]:
%load_ext autoreload
%autoreload 2

What is happening in this notebook?

Use the GEE pipeline to tile up a region, run a model on the chips, and then export the embedding values.
The biggest difference between this and the mining pipeline is the output types. Instead of saving just a float value for chips above a threshold, we'll save an embedding vector for each chip.

The other main difference is that the model is coming from torchgeo rather than from a local keras model.

In [ ]:
from datetime import datetime

import geopandas as gpd

import gee
import utils

In [ ]:
region_name = 'bali'

tile_size = 224*2 # this is the around the max size that GEE exports allow with 12-band imagery
tile_padding = 8

start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 1)
clear_threshold = 0.75

region = gpd.read_file(f'../data/boundaries/{region_name}.geojson').geometry[0].__geo_interface__

tiles = utils.create_tiles(region, tile_size, tile_padding)
print(f"Created {len(tiles):,} tiles")

In [ ]:
data_pipeline = gee.S2_Data_Extractor(
    tiles, 
    start_date, 
    end_date, 
    clear_threshold, 
    batch_size=12 # batch sizes need to be smaller in order to avoid running out of memory
    )
data_pipeline.create_embeddings()